In [58]:
from Bio import Entrez
import pandas as pd
import json
import time
import random
import sys

In [59]:
def search(query):
    Entrez.email = 'kensyrow@naver.com'
    handle = Entrez.esearch(db='pubmed', 
                            sort='relevance', 
                            retmax='3000',
                            retmode='xml', 
                            term=query)
    results = Entrez.read(handle)
    return results

def fetch_details(id_list):
    ids = ','.join(id_list)
    Entrez.email = 'kensyrow@naver.com'
    handle = Entrez.efetch(db='pubmed',
                           retmode='xml',
                           id=ids)
    results = Entrez.read(handle)
    return results

In [60]:
results = search('hasstructuredabstract')
id_list = results['IdList']
papers = fetch_details(id_list)

In [61]:
train_data = []
test_data = []

p = 0
m = 0
r = 0
c = 0

for paper in papers['PubmedArticle']:
    struct = paper['MedlineCitation']['Article']['Abstract']['AbstractText']
    if len(struct) >= 4:
        for content in struct:
            try:
                label = str(content.attributes['Label']).upper()
                
                if label == 'PURPOSE':
                    if p % 10 == 0 :
                        test_data.append((1, "'" + str(content) + "'"))
                    else:
                        train_data.append((1, "'" + str(content) + "'"))
                    p = p + 1
                    
                if label == 'METHODS':
                    if m % 10 == 0 :
                        test_data.append((2, "'" + str(content) + "'"))
                    else:
                        train_data.append((2, "'" + str(content) + "'"))
                    m = m + 1
                    
                if label == 'RESULTS':
                    if r % 10 == 0 :
                        test_data.append((3, "'" + str(content) + "'"))
                    else:
                        train_data.append((3, "'" + str(content) + "'"))
                    r = r + 1
                    
                if label == 'CONCLUSIONS':
                    if c % 10 == 0 :
                        test_data.append((4, "'" + str(content) + "'"))
                    else:
                        train_data.append((4, "'" + str(content) + "'"))
                    c = c + 1
                    
            except:
                pass

In [62]:
print('train data size :', len(train_data))
print('test data size :', len(test_data))

train data size : 5159
test data size : 576


In [63]:
col_names = ['label', 'text']
train_df = pd.DataFrame(train_data, columns=col_names)
train_df = train_df.sort_values(by=['label'], ascending=[True])
train_df.to_csv('pubmed_data/train.csv', header=False, index=False)

In [64]:
test_df = pd.DataFrame(test_data)
test_df.to_csv('pubmed_data/test.csv', header=False, index=False)

## And have to change delimiter to ':::' and delete ' and " at text's head and tail.

### :%s/,/:::
### :%s/'"/'
### :%s/"'/'
### :%s/'/
### :%s/'/